In [1]:
import sys

sys.path.append('..')
from analyses import analysis
from clu import checkpoint
import os

/home/songk/anaconda3/envs/test-2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
workdir = '/data/NFS/potato/songk/spherical-harmonic-net/workdirs/silica-e3schnet-nequip-overfit-jan1'

In [3]:
config, best_state, best_eval_state, metrics_for_best_state = analysis.load_from_workdir(workdir)

In [4]:
metrics_for_best_state

{'train_eval': {'total_loss': array(2.3898318, dtype=float32),
  'focus_and_atom_type_loss': array(1.330384, dtype=float32),
  'position_loss': array(1.0594443, dtype=float32)},
 'val_eval': {'total_loss': array(7.5378923, dtype=float32),
  'focus_and_atom_type_loss': array(3.2127783, dtype=float32),
  'position_loss': array(4.3251233, dtype=float32)},
 'test_eval': {'total_loss': array(8.469297, dtype=float32),
  'focus_and_atom_type_loss': array(3.8207052, dtype=float32),
  'position_loss': array(4.648593, dtype=float32)}}